# Initialization

to run on a different computer, run ``jupyter notebook --ip='*'`` copy and paste address and replace ``localhost`` with ``jmfranck-pi2.syr.edu`` (if on the internet) or ``192.168.1.20`` (if on local network)

In [11]:
%pylab inline
from serial.tools.list_ports import comports, grep
import serial
import time
#import winsound
print serial.VERSION

def generate_beep(freq,duration):
    "Generates a beep -- used winsound.Beep under windows -- use pygame under linux"
    return
def bridge12_wait(s):
    #time.sleep(5)
    def look_for(this_str):
        for j in range(1000):
            a = s.read_until(this_str+'\r\n')
            time.sleep(0.1)
            #print "a is",repr(a)
            print "look for",this_str,"try",j+1
            if this_str in a:
                print "found: ",this_str
                break
    look_for('MPS Started')
    look_for('System Ready')
    return

Populating the interactive namespace from numpy and matplotlib
3.4


print out the com ports, so we know what we're looking for

In [12]:
[(j.device, j.hwid, j.vid, j.description, j.manufacturer) for j in comports()] #list comprehension

[('/dev/ttyACM0',
  'USB VID:PID=2A03:003D SER=75430333137351E03132 LOCATION=1-1.4:1.0',
  10755,
  'Arduino Due Prog. Port',
  'Arduino (www.arduino.org)'),
 ('/dev/ttyAMA0', '3f201000.serial', None, 'ttyAMA0', None)]

Grab the port labeled as Arduino (since the Bridge12 microcontroller is an Arduino)

In [13]:
portlist = [j.device for j in comports() if u'Arduino Due' in j.description]
assert len(portlist)==1
thisport = portlist[0]

# Testing out commands 

try the help command -- start with the simplest way possible, which uses very long wait times:

In [4]:
with serial.Serial(thisport, timeout=1, baudrate=115200) as s:
    time.sleep(20)
    print "done"
    print "initial:",repr(s.read_all()) #Bridge 12 MPS Started
    s.write("help\r") #command for "help"
    time.sleep(10)
    print "after help:",repr(s.read_all()) #read help 

done
initial: 'Bridge12 MPS Started\r\nSystem Ready\r\n'
after help: '\r\nSerial Commands Help\r\n====================\r\n\r\nCommand examples\r\nSet Frequency:\r\nfreq <frequency in kHz>\r\n\r\nQuery Frequency:\r\nfreq?\r\n\r\n\r\nSet Power Level:\r\npower <level in dBm * 10>\r\n\r\nQuery Frequency:\r\npower?\r\n\r\n\r\nFor a complete list see the manual\r\n'


Here, I play with minimal wait times.  Note the timeout=3, which causes readline and read_until to wait longer.  Though there are many lines of code to grab the help response, this should work for any multiline response.

Note that the read_all only reads what's in the buffer *right now* -- with the laptop we were able to call this once, because the bufer fills up so fast.  The USB on the pi seems to be slower.

In [5]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s) #give it time to start
    s.write("help\r") #command for "help"
    print "after help:"
    entire_response = ''
    start = time.time()
    entire_response += s.readline()
    time_to_read_a_line = time.time()-start
    grab_more_lines = True
    while grab_more_lines:
        time.sleep(3*time_to_read_a_line)
        more = s.read_all()
        entire_response += more
        if len(more) == 0:
            grab_more_lines = False
    print repr(entire_response)

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
after help:
'\r\nSerial Commands Help\r\n====================\r\n\r\nCommand examples\r\nSet Frequency:\r\nfreq <frequency in kHz>\r\n\r\nQuery Frequency:\r\nfreq?\r\n\r\n\r\nSet Power Level:\r\npower <level in dBm * 10>\r\n\r\nQuery Frequency:\r\npower?\r\n\r\n\r\nFor a complete list see the manual\r\n'


Now, I play with the waveguide status

In [8]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    def wgstatus_int_singletry():
        s.write('wgstatus?\r')
        return int(s.readline())
    def wgstatus_int():
        "need two consecutive responses that match"
        a = wgstatus_int_singletry()
        b = wgstatus_int_singletry()
        while a != b:
            a = b
            b = wgstatus_int_singletry()
        return a
    bridge12_wait(s)
    print "initial status:",wgstatus_int()
    s.write('wgstatus 1\r')
    print "turn it on:",wgstatus_int()
    time.sleep(3)
    s.write('wgstatus 0\r')
    print "turn it off again:",wgstatus_int()

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
initial status: 0
turn it on: 1
turn it off again: 0


In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    def set_wg(setting):
        """set *and check* waveguide
        
        Parameters
        ==========
        setting: bool
            Waveguide setting appropriate for:
            True: DNP
            False: ESR
        """
        s.write('wgstatus %d\r'%setting)
        for j in range(10):
            s.write('wgstatus?\r')
            result = int(s.readline())
            if result == setting:
                return
        raise RuntimeError("After trying 10 times, I can't get the waveguide to change")
    print "waveguide on"
    set_wg(True)
    time.sleep(5)# just so we're not toggling it back and forth very fast
    print "waveguide off"
    set_wg(False)

# JF stopped here

BEGIN SAMANTHA'S WORK 180226

Frequency query test (new_180226)

In [6]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s) #give it time to start
    s.write("freq?\r") #query for "freq?"
    print "after freq:"
    entire_response = ''
    start = time.time()
    entire_response += s.readline()
    time_to_read_a_line = time.time()-start
    grab_more_lines = True
    while grab_more_lines:
        time.sleep(3*time_to_read_a_line)
        more = s.read_all()
        entire_response += more
        if len(more) == 0:
            grab_more_lines = False
    print repr(entire_response)

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
after freq:
'9500000\r\n'


Frequency runthrough test (new_180226)

In [7]:
freq = linspace(9.4e9,9.9e9, 50)
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s) #give it time to start
    for j,f in enumerate(freq):
        time.sleep(0.5)
        generate_beep(500, 300)
        s.write('freq %.1f\r'%(f/1e3))
    print "after freq:"
    entire_response = ''
    start = time.time()
    entire_response += s.readline()
    time_to_read_a_line = time.time()-start
    grab_more_lines = True
    while grab_more_lines:
        time.sleep(3*time_to_read_a_line)
        more = s.read_all()
        entire_response += more
        if len(more) == 0:
            grab_more_lines = False
    print repr(entire_response)

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
after freq:
''


Test amp status (180226)

In [9]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    def ampstatus_int_singletry():
        s.write('ampstatus?\r')
        return int(s.readline())
    def ampstatus_int():
        "need two consecutive responses that match"
        a = ampstatus_int_singletry()
        b = ampstatus_int_singletry()
        while a != b:
            a = b
            b = wgstatus_int_singletry()
        return a
    bridge12_wait(s)
    print "initial status:",wgstatus_int()
    s.write('ampstatus 1\r')
    print "turn it on:",wgstatus_int()
    time.sleep(3)
    s.write('ampstatus 0\r')
    print "turn it off again:",wgstatus_int()

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
initial status: 0
turn it on: 0
turn it off again: 0


Power Change Test (180226)

In [13]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    def power_int_singletry():
        s.write('power?\r')
        return int(s.readline())
    def power_int():
        "need two consecutive responses that match"
        a = power_int_singletry()
        b = power_int_singletry()
        while a != b:
            a = b
            b = power_int_singletry()
        return a
    bridge12_wait(s)
    print "initial power:",power_int()
    s.write('power 100\r')
    print "up power:",power_int()
    time.sleep(3)
    s.write('power 0\r')
    print "down power:",power_int()

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
initial power: 0
up power: 100
down power: 0


### Initialization for MW power test and Tuning Curve (180306)
    Do I need to choose a variable other than s for the settings for the amplifier and microwave power?-Sam

In [14]:
def wgstatus_int_singletry(s):
    s.write('wgstatus?\r')
    return int(s.readline())
def wgstatus_int(s):
    "need two consecutive responses that match"
    c = wgstatus_int_singletry(s)
    d = wgstatus_int_singletry(s)
    while c != d:
        c = d
        d = wgstatus_int_singletry(s)
    return c
def set_wg(s,setting):
    """set *and check* waveguide

    Parameters
    ==========
    s: Serial object
        gives the connection
    setting: bool
        Waveguide setting appropriate for:
        True: DNP
        False: ESR
    """
    s.write('wgstatus %d\r'%setting)
    for j in range(10):
        result = wgstatus_int(s)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the waveguide to change")
    
def ampstatus_int_singletry(t):
    s.write('ampstatus?\r')
    return int(s.readline())
def ampstatus_int(t):
    "need two consecutive responses that match"
    a = ampstatus_int_singletry(t)
    b = ampstatus_int_singletry(t)
    while a != b:
        a = b
        b = ampstatus_int_singletry(t)
    return a
def set_amp(t,setting):
    """set *and check* amplifier

    Parameters
    ==========
    t: Serial object
        gives the connection
    setting: bool
       Amplifier setting appropriate for:
        True: On
        False: Off
    """
    s.write('ampstatus %d\r'%setting)
    for j in range(10):
        result = ampstatus_int(t)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the amplifier to turn on/off")
    
def rfstatus_int_singletry(u):
    s.write('rfstatus?\r')
    return int(s.readline())
def rfstatus_int(u):
    "need two consecutive responses that match"
    f = rfstatus_int_singletry(u)
    g = rfstatus_int_singletry(u)
    while f != g:
        f = g
        g = rfstatus_int_singletry(u)
    return f
def set_rf(u,setting):
    """set *and check* microwave power

    Parameters
    ==========
    u: Serial object
        gives the connection
    setting: bool
        Microwave Power setting appropriate for:
        True: On
        False: Off
    """
    s.write('rfstatus %d\r'%setting)
    for j in range(10):
        result = rfstatus_int(u)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the mw power to turn on/off")
    
   

Power Initialization value (180306) Do I need the initialization code for setting the power value???

In [ ]:
 def power_int_singletry(v):
    s.write('power?\r')
    return int(s.readline())
def power_int(v):
    "need two consecutive responses that match"
    f = power_int_singletry(v)
    g = power_int_singletry(v)
    while h != i:
        h = i
        i = power_int_singletry(v)
    return f
def set_power(v,setting):
    """set *and check* power

    Parameters
    ==========
    v: Serial object
        gives the connection
    setting: bool
        Power setting appropriate for:
        True: On
        False: Off
    """
    s.write('power %d\r'%setting)
    for j in range(10):
        result = power_int(v)
        if result == setting:
            return
    raise RuntimeError("After checking status 10 times, I can't get the power to change")
    

### Code for MW power test (180306)

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    print "initial wg status:", wgstatus_int(s)
    set_wg(s,True)
    print "initial amp status:", ampstatus_int(t)
    set_amp(t, True)
    print "initial rf status:", rfstatus_int(u)
    set_rf(u, True)
    s.write('power?\r')
    print "initial power:", s.readline()
    s.write('power 100\r')
    time.sleep(10)
    s.write('power 0\r')
    s.write('power?\r')
    print "final power:", s.readline()
    set_rf(u,False)
    set_wg(s,False)

### Tuning Curve Code (180306)

In [ ]:
freq = linspace(9.4e9,9.9e9, 50)
rxvalues = zeros(len(freq))
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    #Turn Components On and Power to 10dBm
    print "initial wg status:", wgstatus_int(s)
    set_wg(s,True)
    print "initial amp status:", ampstatus_int(t)
    set_amp(t, True)
    print "initial rf status:", rfstatus_int(u)
    set_rf(u, True)
    s.write('power?\r')
    print "initial power:", s.readline()
    s.write('power 100\r')
    s.write('power?\r')
    print "power up:", s.readline()
    #FREQUENCY AND RXPOWER SWEEP 
    for j,f in enumerate(freq):
        time.sleep(0.5)
        generate_beep(500, 300)
        s.write('freq %.1f\r'%(f/1e3))
        s.write('rxpowerdbm?\r') 
        rx =s.readline()
        rxvalues[j] = float(rx)/10 
    time.sleep(10)
    #Turn Components Off and Power to 0dBm
    s.write('power 0\r')
    s.write('power?\r')
    print "final power:", s.readline()
    set_rf(u,False)
    set_wg(s,False)
print "j",j, "freq",f, "rxpower", rxvalues #'table' of values
plot(freq, rxvalues) #tuning curve

Original Code from (180226) for MW Power Test

In [16]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    # WG to DNP
    print "initial wg status:",wgstatus_int(s)
    set_wg(s,True)
    #s.write('wgstatus 0\r')
    time.sleep(10)
    set_wg(s,False)
#     #print "turn wg off again:",wgstatus_int()
#     # MW source on
#     print "initial rf status:",rfstatus_int(s)
#     s.write('rfstatus 1\r')
#     print "turn rf on:",rfstatus_int(s)
#     time.sleep(3)
#     # Set forward power to 10 dBm
    
#     #s.write('rfstatus 0\r')
#     #print "turn rf off again:",rfstatus_int()
#     s.write('rfstatus 0\r')
#     print "turn it off again:",rfstatus_int(s)
#     s.write('ampstatus 0\r')
#     print "turn it off again:",ampstatus_int(s)
#     s.write('wgstatus 0\r')
#     print "turn it off again:",wgstatus_int(s)

look for MPS Started try 1
found:  MPS Started
look for System Ready try 1
look for System Ready try 2
look for System Ready try 3
found:  System Ready
initial wg status: 0


Original tuning curve code from (180226)

In [ ]:
freq = linspace(9.4e9,9.9e9, 50)
rxvalues = zeros(len(freq))
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    #WAVEGUIDE TO DNP
    def wgstatus_int_singletry():
        s.write('wgstatus?\r')
        return int(s.readline())
    def wgstatus_int():
        "need two consecutive responses that match"
        c = wgstatus_int_singletry()
        d = wgstatus_int_singletry()
        while c != d:
            c = d
            d = wgstatus_int_singletry()
        return c
    bridge12_wait(s)
    print "initial wg status:",wgstatus_int()
    s.write('wgstatus 1\r')
    print "turn wg on:",wgstatus_int()
    time.sleep(3)
    #s.write('wgstatus 0\r')
    #print "turn wg off again:",wgstatus_int()
    #AMP TO ON
    def ampstatus_int_singletry():
        s.write('ampstatus?\r')
        return int(s.readline())
    def ampstatus_int():
        "need two consecutive responses that match"
        a = ampstatus_int_singletry()
        b = ampstatus_int_singletry()
        while a != b:
            a = b
            b = ampstatus_int_singletry()
        return a
    bridge12_wait(s)
    print "initial amp status:",ampstatus_int()
    s.write('ampstatus 1\r')
    print "turn amp on:",ampstatus_int()
    time.sleep(3)
    #s.write('ampstatus 0\r')
    #print "turn amp off again:",ampstatus_int()
    #MW POWER ON (RFPWER)
    def rfstatus_int_singletry():
        s.write('rfstatus?\r')
        return int(s.readline())
    def rfstatus_int():
        "need two consecutive responses that match"
        f = rfstatus_int_singletry()
        g = rfstatus_int_singletry()
        while f != g:
            f = g
            g = rfstatus_int_singletry()
        return f
    bridge12_wait(s)
    print "initial rf status:",rfstatus_int()
    s.write('rfstatus 1\r')
    print "turn rf on:",rfstatus_int()
    time.sleep(3)
    #s.write('rfstatus 0\r')
    #print "turn rf off again:",rfstatus_int()
    #POWER TO 10dBm
    def power_int_singletry():
        s.write('power?\r')
        return int(s.readline())
    def power_int():
        "need two consecutive responses that match"
        a = power_int_singletry()
        b = power_int_singletry()
        while a != b:
            a = b
            b = power_int_singletry()
        return a
    bridge12_wait(s)
    print "initial power:",power_int()
    s.write('power 100\r')
    print "up power:",power_int()
    time.sleep(3)
    #s.write('power 0\r')
    #print "down power:",power_int()
    #FREQUENCY AND RXPOWER SWEEP 
    for j,f in enumerate(freq):
        time.sleep(0.5)
        generate_beep(500, 300)
        s.write('freq %.1f\r'%(f/1e3))
        s.write('rxpowerdbm?\r') 
        rx =s.readline()
        rxvalues[j] = float(rx)/10 #converting string to float point values
    #ARE THERE PARTS TO THE FREQ CODE I CAN DELETE HERE^^^?
    #POWER TO 0dBm
    s.write('power 0\r')
    print "down power:",power_int()
    time.sleep(3)
    #MW POWER OFF
    s.write('rfstatus 0\r')
    print "turn rf off again:",rfstatus_int()
    time.sleep(3)
    #WG TO ESR MODE
    s.write('wgstatus 0\r')
    print "turn wg off again:",wgstatus_int()
    time.sleep(3)
print "j",j, "freq",f, "rxpower", rxvalues #'table' of values
plot(freq, rxvalues) #tuning curve

freq query test

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    print "done"
    generate_beep(1000,300)
    s.write('freq?\r')
    print float(s.readline())
print "final"

frequency test

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    #time.sleep(10)
    tmp = s.read_all()
    print repr(tmp)
    print "done"
    generate_beep(1000,300)
    for freq in linspace(9.45e9,9.51e9,11):
        time.sleep(0.5)
        generate_beep(500,300)
        s.write('freq %.1f\r'%(freq/1e3))
    generate_beep(1000,300)
    time.sleep(5)
    print "final:", s.read_all()

preliminary code for ampstatus and freq (no need to run):

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    bridge12_wait(s)
    print "done"
    print "initial:",s.readline()
    s.write('ampstatus 0\r') #command amp to be OFF
    s.write('ampstatus?\r') #command to check amp status (ON/OFF)
    amp_status = int(s.readline()) #read amp status as an integer
    print "amp status:",amp_status #print amp status 
    generate_beep(1000, 300) #signal to begin running through frequencies
    for mw_freq in linspace(9.45e9,9.51e9,11): #frequencies being ran through 
        time.sleep(0.5) #time between frequencies
        generate_beep(500, 300) #signal frequency change
        s.write('freq %.1f\r'%(mw_freq/1e3)) #changing frequency values to a string for Bridge12 to read
    generate_beep(1000, 300) #signal when finished running through frequencies

We want functions that will change a setting and verify, so let's do that here for the waveguide.  Check this out and see if you get the basic idea.

playing with loops

In [ ]:
for mw_freq in linspace(9.5e9,9.51e9,11):
    print 'freq %.1f\r'%(mw_freq/1e3) #printing

just checking to see if sounds work

In [ ]:
import winsound
duration = 1000  # millisecond
freq = 440  # Hz
generate_beep(880, 300)

use this block for the tuning curve:

In [ ]:
freq = linspace(9.4e9,9.9e9, 50)
rxvalues = zeros(len(freq))
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    time.sleep(10)
    print "initial:",s.readline()
    #ampstatus commands
    s.write('ampstatus 0\r')
    time.sleep(0.5)
    s.write('ampstatus?\r')
    amp_status = int(s.readline())
    print "amp status:",amp_status
    time.sleep(5)
    generate_beep(1000,300)
    #frequency and rxpower
    for j,f in enumerate(freq):
        time.sleep(0.5)
        generate_beep(500, 300)
        s.write('freq %.1f\r'%(f/1e3)) #frequencies are now defined as f, NOT mw_freq
        s.write('rxpowerdbm?\r') 
        rx =s.readline()
        rxvalues[j] = float(rx)/10 #converting string to float point values
    generate_beep(1000, 300)
    time.sleep(5)
    print "final:",s.read_all()
print "j",j, "freq",f, "rxpower", rxvalues #'table' of values
plot(freq, rxvalues) #tuning curve

another option for tuning curve (with sound):

In [ ]:
freq = linspace(9.4e9,9.9e9, 50)
rxvalues = zeros(len(freq))
txvalues = zeros(len(freq))
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    time.sleep(10) 
    print "done."
    print "initial:",s.readline()
    s.write('power?\r')
    check_set_power = float(s.readline())/10.
    print "power is set to",check_set_power
    s.write('ampstatus 0\r') #command amp to be OFF
    s.write('ampstatus?\r') #command to check amp status (ON/OFF)
    amp_status = int(s.readline()) #read amp status as an integer
    print "amp status:",amp_status #print amp status 
    s.write('rfstatus?\r') #command to check rf status (ON/OFF)
    rf_status = int(s.readline()) #read rf status as an integer
    print "rf status:",rf_status #print amp status 
    s.write('wgstatus 1\r') #switch to DNP mode
    s.read_all()
    #s.write('wgstatus?\r') #check wg status
    #wg_status1= int(s.readline()) #read wg status as an integer
    #print "wg status1:", wg_status1 #print wg status
    generate_beep(1000,300)
    set_power = 10.0
    s.write('power %d\r'%(round(set_power*10)))
    s.write('power?\r')
    check_set_power = float(s.readline())/10.
    print "power is set to",check_set_power
    generate_beep(1000, 300) #signal to begin running through frequencies
    for j,f in enumerate(freq):
        time.sleep(0.5) #time between frequencies
        generate_beep(500, 300) #signal frequency change
        s.write('freq %.1f\r'%(mw_freq/1e3)) #changing frequency values to a string for Bridge12 to read
        s.write('rxpowerdbm?\r')
        rxvalues[j]=float(s.readline())
        s.write('txpowerdbm?\r')
        txvalues[j]=float(s.readline())
    generate_beep(1000, 300) #signal when finished running through frequencies
    time.sleep(5) #time to process
    s.write('wgstatus 0\r')
    #s.write('wgstatus?\r')
    #wg_status0 = int(s.readline())
    print "wg status0:", wg_status0
    print "final:",s.read_all() #print any values coming out from "with" loop
plot(freq, rxvalues, alpha=0.5, label='Rx')
plot(freq, txvalues, alpha=0.5, label='Tx')
legend()

testing the power

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    s.read_until('Started\r\n') #read this instead of time sleep
    s.write('power 0\r')
    time.sleep(10)

testing the waveguide switch

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    s.read_until('Started\r\n')
    #s.write('power 100\r')
    for j in range(10):
        time.sleep(5)
        s.write('wgstatus?\r')
        print "wgstatus:",repr(s.readline())
    #print "power:", int(s.readline())

testing frequency

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    s.read_until('Started\r\n')
    time.sleep(10)
    generate_beep(700,300)
    s.write('freq 9505000.0/r')
    generate_beep(500,300)
print "done"

testing both power and rfstatus

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    s.read_until('Started\r\n')
    s.write('power 0\r')
    s.write('power?\r')
    print "power:", int(s.readline())
    generate_beep(1000,300)
    s.write('rfstatus 1\r')
    s.write('rfstatus?\r')
    print "rfstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(10)
print "done"

messing with lots of on/off commands at once

In [ ]:
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    s.read_until('Started\r\n')
    s.write('power 0\r')
    s.write('power?\r')
    print "power:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('wgstatus 1\r')
    s.write('wgstatus?\r')
    print "wgstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('ampstatus 1\r')
    s.write('ampstatus?\r')
    print "ampstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('rfstatus 1\r')
    s.write('rfstatus?\r')
    print "rfstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(10)
    s.write('rfstatus 0\r')
    s.write('rfstatus?\r')
    print "rfstatus:" int(s.readline)
    s.write('ampstatus 0\r')
    s.write('ampstatus?\r')
    print "ampstatus:" int(s.readline())
print "done"

will be a fully operational tuning curve with power on/off

In [ ]:
freq = linspace(9.845e9,9.855e9, 50)
rxvalues = zeros(len(freq))
with serial.Serial(thisport, timeout=3, baudrate=115200) as s:
    s.read_untill('Started\r\n')
    s.write('power 100\r')
    s.write('power?\r')
    print "power:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('wgstatus 1\r')
    s.write('wgstatus?\r')
    print "wgstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('ampstatus 1\r')
    s.write('ampstatus?\r')
    print "ampstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('rfstatus 1\r')
    s.write('rfstatus?\r')
    print "rfstatus:", int(s.readline())
    generate_beep(1000,300)
    for j, f in enumerate(freq): 
        time.sleep(0.5) 
        generate_beep(500, 300) 
        s.write('freq %.3f\r'%(freq/1e3)) 
        s.write('rxpowerdbm?\r')
        rxvalues[j]=float(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('rfstatus 0\r')
    s.write('rfstatus?\r')
    print "rfstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('ampstatus 0\r')
    s.write('ampstatus?\r')
    print "ampstatus:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
    s.write('power 0\r')
    s.write('power?\r')
    print "power:", int(s.readline())
    generate_beep(1000,300)
    time.sleep(5)
plot(freq, rxvalues, alpha=1.0, label='Rx')
legend()